<a href="https://colab.research.google.com/github/Saumya-Chauhan-MHC/PyTorch-Resnet18_CIFAR10_Finetune_Modularize/blob/main/PyTorch_Resnet18_CIFAR10_Finetune_Modularize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the required libraries: Make sure Wandb is installed. Install it using pip:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [2]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch.optim as optim
from data_loader import get_data_loaders
from models import get_model
import torch
import torch.nn as nn

In [4]:
wandb.init(project='cifar-10_resnet18_pretrained_finetune_modularize_no_lightning', entity='saumya-s-chau')

wandb: Currently logged in as: saumya-s-chau. Use `wandb login --relogin` to force relogin


In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
# Get the model
model = get_model(len(classes))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 81.3MB/s]


In [8]:
# Define hyperparameters
batch_size = 64
num_workers = 2
lr = 0.01
momentum = 0.9
weight_decay = 5e-4
epochs = 10

In [9]:
# Get data loaders
trainloader, testloader = get_data_loaders(batch_size=batch_size, num_workers=num_workers)

100%|██████████| 170498071/170498071 [00:04<00:00, 42420989.64it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [12]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        # inputs, targets = inputs, targets
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    train_loss /= len(trainloader)
    train_accuracy = 100.0 * correct / total

    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            # inputs, targets = inputs.to('cuda'), targets.to('cuda')
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        test_loss /= len(testloader)
        test_accuracy = 100.0 * correct / total

        # Log the metrics to Wandb
        wandb.log({"epoch": epoch, "train_loss": train_loss, "train_accuracy": train_accuracy, 
                   "test_loss": test_loss, "test_accuracy": test_accuracy})

        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.2f}%"
              f" - Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.2f}%")

wandb.finish()


Epoch 1/10 - Train Loss: 0.8256 - Train Accuracy: 71.68% - Test Loss: 0.7088 - Test Accuracy: 75.93%
Epoch 2/10 - Train Loss: 0.7420 - Train Accuracy: 74.88% - Test Loss: 0.7831 - Test Accuracy: 74.45%
Epoch 3/10 - Train Loss: 0.6542 - Train Accuracy: 77.82% - Test Loss: 0.6696 - Test Accuracy: 77.62%
Epoch 4/10 - Train Loss: 0.6131 - Train Accuracy: 79.05% - Test Loss: 0.6032 - Test Accuracy: 79.34%
Epoch 5/10 - Train Loss: 0.5958 - Train Accuracy: 79.72% - Test Loss: 0.6439 - Test Accuracy: 78.45%
Epoch 6/10 - Train Loss: 0.5713 - Train Accuracy: 80.39% - Test Loss: 0.6370 - Test Accuracy: 78.39%
Epoch 7/10 - Train Loss: 0.5416 - Train Accuracy: 81.25% - Test Loss: 0.5646 - Test Accuracy: 80.67%
Epoch 8/10 - Train Loss: 0.5209 - Train Accuracy: 81.97% - Test Loss: 0.5457 - Test Accuracy: 81.36%
Epoch 9/10 - Train Loss: 0.5062 - Train Accuracy: 82.71% - Test Loss: 0.6043 - Test Accuracy: 79.28%
Epoch 10/10 - Train Loss: 0.4961 - Train Accuracy: 82.98% - Test Loss: 0.5215 - Test Accura

epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▂▁▄▅▅▅▇▇▅█
test_loss,▆█▅▃▄▄▂▂▃▁
train_accuracy,▁▃▅▆▆▆▇▇██
train_loss,█▆▄▃▃▃▂▂▁▁
epoch,9
test_accuracy,82.33
test_loss,0.52153
train_accuracy,82.982
train_loss,0.49609
